In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import scipy.stats as stats
import bspline
import bspline.splinelab as splinelab



In [ ]:
data=pd.read_csv("Assets.csv")
# ['Year', 'Total Assets', 'Log Return', 'Yearly Volatility','Quarterly Volatility', 'Distress Barrier', 'Moody\'s Ratings', 'Past RNPD', 'Normalized Past RNPD']
# print(data.head)

In [ ]:
# dataset=pd.DataFrame([], columns=['Year', 'Total Assets', 'Distress Barrier', 'Log Return', 'Yearly Volatility', 'Quarter Volatility'])

# start_year=2001
# end_year=2022

# dataset['Year']=np.arange(start_year, end_year+1)
# dataset['Total Assets']= pd.Series([71753078142.00, 73535346705.00, 78235030416.00, 87355506463.00, 105912000000.00, 114571000000.00, 113550000000.00, 120751000000.00, 117451000000.00, 116042000000.00, 110884000000.00, 107309000000.00, 116030000000.00, 139673000000.00, 154796000000.00, 161894000000.00, 208396000000.00, 209056000000.00, 189460000000.00, 203829000000.00, 234233000000.00, 223807000000.00], dtype='float')
# dataset['Distress Barrier']=pd.Series([15669733148, 16622163564, 17776465778, 17833508474, 16877445036, 18517386840, 21442068690, 23291326752, 24542066485, 28766631090, 29199892732, 30006391296, 29813639417, 32344507489, 34364836658, 37469268824, 46353699857, 50017932700, 54655855964, 57011267289, 64287512856, 69763695000], dtype='float')
# for t in range(1, end_year+1-start_year):
#     dataset.iloc[t,3]=np.log(dataset.iloc[t,1] / dataset.iloc[t-1, 1]) #filling Log return

# dataset['Log Return']= dataset['Log Return'].astype('float')
# for i in range(2, end_year+1-start_year):
#     returns=np.array(dataset.iloc[1:i+1, 3])
#     yrly_vol=np.std(a=returns, dtype='float', ddof=1)*(np.sqrt(261))
#     dataset.iloc[i, 4]=yrly_vol
#     dataset.iloc[i, 5]=yrly_vol/np.sqrt(4)

In [ ]:
output=pd.DataFrame([], columns=['Year', 'Total Assets', 'Distress Barrier', 'Past RNPD', "Def Probability", 'Moody\'s Ratings', 'Normalized Past RNPD'])
# later add min max normalized Default Probability

creating dataset

In [ ]:
# dataset=pd.DataFrame([], columns=['Year', 'Total Assets', 'Distress Barrier', 'Log Return', 'Yearly Volatility', 'Quarter Volatility'])

# start_year=2001
# end_year=2022

# dataset['Year']=np.arange(start_year, end_year+1)
# dataset['Total Assets']= pd.Series([71753078142.00, 73535346705.00, 78235030416.00, 87355506463.00, 105912000000.00, 114571000000.00, 113550000000.00, 120751000000.00, 117451000000.00, 116042000000.00, 110884000000.00, 107309000000.00, 116030000000.00, 139673000000.00, 154796000000.00, 161894000000.00, 208396000000.00, 209056000000.00, 189460000000.00, 203829000000.00, 234233000000.00, 223807000000.00], dtype='float')
# dataset['Distress Barrier']=pd.Series([15669733148, 16622163564, 17776465778, 17833508474, 16877445036, 18517386840, 21442068690, 23291326752, 24542066485, 28766631090, 29199892732, 30006391296, 29813639417, 32344507489, 34364836658, 37469268824, 46353699857, 50017932700, 54655855964, 57011267289, 64287512856, 69763695000], dtype='float')
# for t in range(1, end_year+1-start_year):
#     dataset.iloc[t,3]=np.log(dataset.iloc[t,1] / dataset.iloc[t-1, 1]) #filling Log return

# dataset['Log Return']= dataset['Log Return'].astype('float')
# for i in range(2, end_year+1-start_year):
#     returns=np.array(dataset.iloc[1:i+1, 3])
#     yrly_vol=np.std(a=returns, dtype='float', ddof=1)*(np.sqrt(261))
#     dataset.iloc[i, 4]=yrly_vol
#     dataset.iloc[i, 5]=yrly_vol/np.sqrt(4)

In [ ]:
new_mat_og=pd.DataFrame([], columns=['Year', 'Total Assets', 'Distress Barrier', 'Volatility - Year', 'Volatility - Quarter', 'Quarter Number', 'N(d1)', 'd1', 'd2', 'N(-d2)', 'Moody\'s Rating'])
# we will have T+1 values per year, per simulation, qtr num onwards all unique

In [ ]:
mu = 0.005   # drift
r = 0.124     # risk-free rate
T = 4
M = 1         # maturity
N_MC = 2500    # number of paths
risk_lambda = 0.6
 # risk aversion parameter
delta_t = M / T                # time interval
gamma = np.exp(- r * delta_t)  # discount factor
# Define the risk aversion parameter
reg_param = 1e-3

# To get meaninful results, one should have ncolloc >= p+1
p = 4 # order of spline (as-is; 3: cubic, 4: B-spline?)
ncolloc = 12

num_t_steps = T + 1
num_basis =  ncolloc

year_avg=np.arange(0.0, 21.0)


In [ ]:
# standard normal random numbers
RN = pd.DataFrame(np.random.randn(N_MC,T), index=range(1, N_MC+1), columns=range(1, T+1))

In [ ]:
def terminal_payoff(ST, K):
    # ST   final stock price    # K    strike
    payoff = max(K - ST, 0)
    return payoff


# functions to compute optimal hedges
def function_A_vec(t,delta_S_hat,data_mat_t):
    # Compute the matrix A_{nm} from Eq. (52) (with a regularization!)
    X_mat = data_mat_t[t,:,:]
    num_basis_funcs = X_mat.shape[1]
    this_dS = delta_S_hat.loc[:,t].values
    hat_dS2 = (this_dS**2).reshape(-1,1)
    A_mat = np.dot(X_mat.T, X_mat * hat_dS2) + reg_param * np.eye(num_basis_funcs)
    return A_mat


def function_B_vec(t, Pi_hat, delta_S_hat, S, data_mat_t):
    coef = 1.0/(2 * gamma * risk_lambda)
    tmp =  Pi_hat.loc[:,t+1] * delta_S_hat.loc[:,t] + coef * (np.exp(mu*delta_t) - np.exp(r*delta_t))* S.loc[:,t]
    X_mat = data_mat_t[t,:,:]  # matrix of dimension N_MC x num_basis
    B = np.dot(X_mat.T, tmp)
    return B


In [ ]:
for i in range(2, 22): #use this to mean from 2002 to 2022
    count=i
    curr_yr_avg=np.arange(0.0, T+2.0)

    curr_year=i+2001

    print("\nYear ", curr_year, ". Simulation", i-1,"/20.")

    year=data.iloc[i, 0]
    S0=data.iloc[i, 1]                 #initial stock price
    S0=S0/1000000000
    sigma=data.iloc[i, 3]              # yearly volatility
    sigma_qtr=data.iloc[i, 4]          # quarterly volatility
    K=data.iloc[i, 5]
    K=K/1000000000
    moody=data.iloc[i, 6]

    # # standard normal random numbers
    # RN = pd.DataFrame(np.random.randn(N_MC,T), index=range(1, N_MC+1), columns=range(1, T+1))

    # print("year: ", year)
    # print("S0: ", S0)
    # print("sigma: ", sigma)
    # print("sigma_qtr: ", sigma_qtr)
    # print("K: ", K)

    S = pd.DataFrame([], index=range(1, N_MC+1), columns=range(T+1))
    S.loc[:,0] = S0
    # print("S\n", S.head)

    for t in range(1, T+1):
        S.loc[:,t] = S.loc[:,t-1] * np.exp((mu - 1/2 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * RN.loc[:,t])

    # print("S\n", S.head)

    delta_S = S.loc[:,1:T].values - np.exp(r * delta_t) * S.loc[:,0:T-1]
    delta_S_hat = delta_S.apply(lambda x: x - np.mean(x), axis=0)

    # state variable
    X = - (mu - 1/2 * sigma**2) * np.arange(T+1) * delta_t + np.log(S.astype(float) / 1.0)  # delta_t here is due to their conventions
    # print("X\n", X.head)

    X_min = np.min(np.min(X))
    X_max = np.max(np.max(X))

    tau = np.linspace(X_min, X_max, ncolloc)  # These are the sites to which we would like to interpolate

    # k is a knot vector that adds endpoints repeats as appropriate for a spline of order p
    k = splinelab.aptknt(tau, p)
    # print("k: ", k)

    # Spline basis of order p on knots k
    basis = bspline.Bspline(k, p)
    f = plt.figure()
    # basis.plot()

    # Make data matrices with feature values
    data_mat_t = np.zeros((num_t_steps, N_MC, num_basis))

    # fill it
    for i in np.arange(num_t_steps):
        x = X.values[:,i]
        basis_arr=np.array([ basis(i) for i in x ])
        data_mat_t[i,:,:] = basis_arr

    # Compute optimal hedge and portfolio value
    # portfolio value
    Pi = pd.DataFrame([], index=range(1, N_MC+1), columns=range(T+1))
    Pi.iloc[:,-1] = S.iloc[:,-1].apply(lambda x: terminal_payoff(x, K))

    Pi_hat = pd.DataFrame([], index=range(1, N_MC+1), columns=range(T+1))
    Pi_hat.iloc[:,-1] = Pi.iloc[:,-1] - np.mean(Pi.iloc[:,-1])

    # optimal hedge
    a = pd.DataFrame([], index=range(1, N_MC+1), columns=range(T+1))

    a.iloc[:,-1] = 0

    for t in range(T-1, -1, -1):

        A_mat = function_A_vec(t, delta_S_hat, data_mat_t)
        B_vec = function_B_vec(t, Pi_hat, delta_S_hat, S, data_mat_t)

        # Convert A_mat and B_vec to a NumPy array of floats
        A_mat = np.array(A_mat, dtype=float)
        B_vec = np.array(B_vec, dtype=float)

        phi = np.dot(np.linalg.inv(A_mat), B_vec)
        a.loc[:,t] = np.dot(data_mat_t[t,:,:],phi)
        Pi.loc[:,t] = gamma * (Pi.loc[:,t+1] - a.loc[:,t] * delta_S.loc[:,t])
        Pi_hat.loc[:,t] = Pi.loc[:,t] - np.mean(Pi.loc[:,t])

    # print("a\n", a.head)
    # print("Pi\n", Pi.head)

    a_avg=a.mean()


    # using Nd1 as is, 5 data points
    Nd1_og=a_avg.astype(float)
    # using zscore to get from Nd1 to d1
    
    d1_og = stats.zscore(Nd1_og)
    # print("D1:\n",d1_og)e



    # where T is the total number of timesteps
    # the variable qtr is an iterator over the Timesteps, starting from O to T
    # so it runs for a total of T+1 times (it T =2, qtr=[0,1,2])
    for qtr in range(0,T+1): #in range(a,b), a in inclusive and b is exclusive
        # print(qtr)
        
        this_Nd1_og=Nd1_og[qtr]
        this_d1_og=d1_og[qtr]

        d2_og=this_d1_og-(sigma_qtr*np.sqrt((T-qtr)/T))
        # print("d2_og: ", d2_og)

        Nd2_og=norm.cdf(-d2_og)
        # print("N(-d2_og): ", Nd2_og)


        # LCL = (S0*this_Nd1_og)-(K*Nposd2_og*np.exp(r*T))

        new_row_og =[year, S0, K, sigma, sigma_qtr, qtr, Nd1_og[qtr], d1_og[qtr], d2_og, Nd2_og, moody]
        # print(new_row_og)
        new_mat_og.loc[len(new_mat_og)] = new_row_og
        curr_yr_avg[qtr]=Nd2_og
        # print(curr_yr_avg[qtr])
    
    sum=0
    for i in range(0,T+1):
        sum+=curr_yr_avg[i]
    yr_avg=sum/T

    # data = ['Year', 'Total Assets', 'Log Return', 'Yearly Volatility','Quarterly Volatility', 'Distress Barrier', 'Moody\'s Ratings', 'Past RNPD', 'Normalized Past RNPD']
    #output = 'Year', 'Total Assets', 'Distress Barrier', 'Past RNPD', "Def Probability", 'Moody\'s Ratings', 'Normalized Past RNPD'])
    new_output_row=[year, data.iloc[count, 1],  data.iloc[count, 5], data.iloc[count, 7], yr_avg, data.iloc[count, 6], data.iloc[count, 8]]
    output.loc[len(output)] = new_output_row
    # print("\nloop over")



In [ ]:
# add min max norm of def probability

# Calculate min and max of 'Def Probability'
min_prob = output['Def Probability'].min()
max_prob = output['Def Probability'].max()

# Create a new column with normalized values
output['Normalized Def Probability'] = (output['Def Probability'] - min_prob) / (max_prob - min_prob)

us=output['Normalized Def Probability']
moody=output['Moody\'s Ratings']
exesum=output['Normalized Past RNPD']
output['EucDist - Moody vs Us'] = (np.sqrt((us-moody)**2))
output['EucDist - Moody vs ExeSum'] = (np.sqrt((exesum-moody)**2))
output['EucDist - Us vs ExeSum'] = (np.sqrt((us-exesum)**2))

output["Compare RNPD"] = [0.6305035, 0.63923975,	0.61812033,	0.65817712,	0.71738401,	0.78523653,	0.79311,	0.80536048,	0.83774903,	0.8355024,	0.79626415,	0.82442427,	0.74055703,	0.70127537,	0.65587051,	0.71405749,	0.85471027,	0.75278117,	0.76817475,	0.84146686]
output["Compare Norm RNPD"] = [0.05234022, 0.08926593, 0,         0.16930893,  0.41956002, 0.70635379, 0.73963276, 0.79141214, 0.92830955, 0.91881367, 0.75296448, 0.87198949, 0.51750594, 0.35147327, 0.15955953, 0.40549974, 1,         0.56917399, 0.63423838, 0.94402378]


In [ ]:
# print("Original\n", new_mat_og)

In [ ]:
import scipy.stats as stats
import numpy as np

# Parameters for the normal distribution
mean = 0
std_dev = 1

# Probability (quantile)
prob = -0.03

# Calculate the inverse CDF (95th percentile)
value = stats.norm.ppf(prob, loc=mean, scale=std_dev)
print(value)


In [ ]:
# print(data.head())

In [ ]:
print(output)